In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ieee-fraud-detection/sample_submission.csv
/kaggle/input/ieee-fraud-detection/test_identity.csv
/kaggle/input/ieee-fraud-detection/train_identity.csv
/kaggle/input/ieee-fraud-detection/test_transaction.csv
/kaggle/input/ieee-fraud-detection/train_transaction.csv


In [2]:
test_identity = pd.read_csv("/kaggle/input/ieee-fraud-detection/test_identity.csv")
test_transaction = pd.read_csv("/kaggle/input/ieee-fraud-detection/test_transaction.csv")

In [3]:
test = test_transaction.merge(test_identity, on='TransactionID', how='left')
test.columns = test.columns.str.replace(r'^id-(\d+)', r'id_\1', regex=True)

In [4]:
!pip install dagshub mlflow

In [5]:
import dagshub
dagshub.init(repo_owner='giorgitorro', repo_name='FraudDetection_gtoro22', mlflow=True)

Accessing as giorgitorro

Initialized MLflow to track repo "giorgitorro/FraudDetection_gtoro22"

Repository giorgitorro/FraudDetection_gtoro22 initialized!

In [7]:
import mlflow
logged_model = 'runs:/9ce7561ee59240fbbec41b242a3556d4/rf_pipeline_model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)



2025/04/29 16:23:51 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - imbalanced-learn (current: 0.12.3, required: imbalanced-learn==0.13.0)
 - scikit-learn (current: 1.5.2, required: scikit-learn==1.6.1)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.6.1 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeCla

In [8]:
y_test_pred_proba = loaded_model.predict(test)


submission = pd.read_csv("/kaggle/input/ieee-fraud-detection/sample_submission.csv")

# Create submission DataFrame
submission = pd.DataFrame({
    'TransactionID': test['TransactionID'],
    'isFraud': y_test_pred_proba
})

# Save to CSV
submission.to_csv('submission.csv', index=False)

# Preview
print(submission.head())


Columns with NaNs after transformation: ['P_emaildomain_woe', 'id_30_woe', 'id_31_woe', 'id_33_woe', 'DeviceInfo_woe']
   TransactionID  isFraud
0        3663549        0
1        3663550        0
2        3663551        0
3        3663552        0
4        3663553        0
